Парсер сайта napopravku.ru.

Собирает данные о клиниках:

- базовую информацию (адрес, телефон, время работы)
- тип клиники
- описание и преимущества
- среднюю оценку и количество отзывов
- информацию о врачах
- цены

Часть первая. Собирает ссылки страниц клиник.

Последние правки: 2021-12-27

## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
# тестовая страница для Селениума

test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/proxy_list.txt'

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]

In [ ]:
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси проще прописать вручную, ввод со строки постоянно глючит

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

#proxy_number = 1

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки ##

На сайте простая пагинация, с 1 и далее. На странице по 20 клиник.

In [ ]:
# начальные ссылки

#url_part_1 = 'https://napopravku.ru/moskva/stomatologia/page-'

#url_part_1 = 'https://spb.napopravku.ru/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/nsk/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/ekb/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/kazan/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/nn/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/chelyabinsk/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/samara/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/rostov-na-donu/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/ufa/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/krasnoyarsk/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/voronezh/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/perm/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/volgograd/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/krasnodar/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/saratov/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/tolyatti/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/barnaul/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/izhevsk/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/ulyanovsk/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/vladivostok/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/yaroslavl/stomatologia/page-'

#url_part_1 = 'https://napopravku.ru/irkutsk/stomatologia/page-'

url_part_1 = 'https://napopravku.ru/tyumen/stomatologia/page-'

In [ ]:
# открываю страницу, чтобы посчитать общее количество клиник

driver.get(url_part_1 + '1/')

In [ ]:
# количество клиник

clinic_number = int(re.sub(r'\D', '',
                           driver.find_element_by_class_name('selection-info__result').text))

page_number = math.ceil(clinic_number / 20)

print('Количество клиник: ' + str(clinic_number))
print('Количество страниц: ' + str(page_number))

### Код парсера ###

На этом же этапе собираю теги клиники, потому на странице, они иногда "теряются".

In [ ]:
data = []

for i in tqdm(range(1, page_number + 1)):
    driver.get(url_part_1 + str(i) + '/')
    sleep(3)

    object_block = driver.find_elements_by_class_name('object-info') # вытаскиваю блок с заголовком

    for item in object_block: # вытаскиваю название клиники и ссылку на страницу
            clinic = item.find_element_by_class_name('object-info__title').text
            clinic_page = (item.find_element_by_class_name('object-info__title')
                           .find_element_by_tag_name('a').get_attribute('href'))

            
            clinic_tags = [] # вытаскиваю теги
            try:
                clinic_tag_block = item.find_elements_by_class_name('object-info__chip')
                for tag in clinic_tag_block:
                    clinic_tags.append(tag.find_element_by_tag_name('a')
                                       .get_attribute('textContent')
                                       .replace('\n', '').strip())


            except:
                pass


            data.append([clinic, clinic_page, clinic_tags])

In [ ]:
# записываю данные в датафрейм
head_list = ['clinic', 'clinic_page', 'clinic_tags']

work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.shape

In [ ]:
work.sample(3)

In [ ]:
work.to_excel(file_path + 'napopravku_links_irkutsk.xlsx', index=False)

In [ ]:
driver.close()